In [ ]:
# Ipython
from sympy.physics.vector import init_vprinting
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
get_ipython().run_line_magic('matplotlib', 'inline')
get_ipython().run_line_magic('load_ext', 'autoreload')
get_ipython().run_line_magic('autoreload', '2')
init_vprinting(use_latex='mathjax', pretty_print=False)
# yams
from welib.yams.models.OneRigidBody_sympy import *  # get_model_one_body, x, phi_x, xd, omega_x, x_BG, Jxx_B, M_B, etc
from welib.yams.yams_sympy_tools import *
from welib.yams.yams_sympy_model import *

In [ ]:
exprHasFunction(x);
x

y
z


In [ ]:
model = get_model_one_body('B100010', linRot=False, orderMM=1, orderH=1, 
                           J_cross=False, CG_on_z=True, J_at_Origin=True)
body = model.body
ref  = model.ref
z_BB= symbols('z_BB')
z_BM= symbols('z_BM')
z_B0= symbols('z_B0')
K_Mx, K_My, K_Mz          = symbols('K_x_M, K_y_M, K_z_M') # Mooring restoring
K_Mphix, K_Mphiy, K_Mphiz = symbols('K_phi_x_M, K_phi_y_M, K_phi_z_M') # Mooring restoring
C_Mx, C_My, C_Mz          = symbols('C_x_M, C_y_M, C_z_M') # Mooring restoring
C_Mphix, C_Mphiy, C_Mphiz = symbols('C_phi_x_M, C_phi_y_M, C_phi_z_M') # Mooring restoring
# Buoyancy force
P_B = body.origin.locatenew('P_B', z_BB * body.frame.z) # <<<< Measured from T
P_M = body.origin.locatenew('P_M', z_BM * body.frame.z) # <<<< Measured from T
P_0 = body.origin.locatenew('P_0', z_B0 * body.frame.z) # <<<< Measured from T
P_O = body.origin
#P_B = body.origin
P_M = body.origin
#F_B = Function('F_B')(dynamicsymbols._t, phi_y)
F_B = dynamicsymbols('F_B') # Buoyancy force, function to time
#F_B = Function('F_B')(dynamicsymbols._t, phi_y)
model.addForce(body,  P_B,        F_B * ref.frame.z)

#P_B = body.origin
P_M = body.origin
F_hx = dynamicsymbols('F_hx') # Buoyancy force, function to time 
F_hy = dynamicsymbols('F_hy') # Buoyancy force, function to time 
F_hz = dynamicsymbols('F_hz') # Buoyancy force, function to time 
M_hx = dynamicsymbols('M_hx') # Buoyancy force, function to time 
M_hy = dynamicsymbols('M_hy') # Buoyancy force, function to time 
M_hz = dynamicsymbols('M_hz') # Buoyancy force, function to time 

model.addForce(body,  P_0,        F_hx * ref.frame.x + F_hy * ref.frame.y + F_hz * ref.frame.z)
model.addMoment(body, body.frame, M_hx * ref.frame.x + M_hy * ref.frame.y + M_hz * ref.frame.z)

addRestoring=True
fr=0; Mr=0;
if addRestoring:
    fr += -K_Mx * x *ref.frame.x #if bDOFs[0] else 0
    fr += -K_My * y *ref.frame.y #if bDOFs[1] else 0
    fr += -K_Mz * z *ref.frame.z #if bDOFs[2] else 0
    fr += -C_Mx * x.diff(dynamicsymbols._t) *ref.frame.x #if bDOFs[0] else 0
    fr += -C_My * y.diff(dynamicsymbols._t) *ref.frame.y #if bDOFs[1] else 0
    fr += -C_Mz * z.diff(dynamicsymbols._t) *ref.frame.z #if bDOFs[2] else 0
    Mr += -K_Mphix * phi_x *ref.frame.x #if bDOFs[3] else 0
    Mr += -K_Mphiy * phi_y *ref.frame.y #if bDOFs[4] else 0
    Mr += -K_Mphiz * phi_z *ref.frame.z #if bDOFs[5] else 0
    Mr += -C_Mphix * phi_x.diff(dynamicsymbols._t) *ref.frame.x #if bDOFs[3] else 0
    Mr += -C_Mphiy * phi_y.diff(dynamicsymbols._t) *ref.frame.y #if bDOFs[4] else 0
    Mr += -C_Mphiz * phi_z.diff(dynamicsymbols._t) *ref.frame.z #if bDOFs[5] else 0
    model.addForce(body, P_M,  fr)
    model.addMoment(body, body.frame, Mr)

model.kdeqsSubs
model.coordinates
#model.body.inertia= (inertia(model.body.frame, J_O, J_O, J_zz), model.body.origin)
# --- Compute Kane's equations of motion
print('>>> Computing equations of motion...')
model.kaneEquations(Mform='TaylorExpanded')
extraSubs=[]
#extraSubs+=[(omega_z,0),(omega_x,0)]
extraSubs+=[(omega_x,0)]
print('Extra Subs:')
extraSubs
#EOM = model.to_EOM(extraSubs=extraSubs)
#EOM.mass_forcing_form()
#EOM.EOM
#EOM.M
#EOM.F
#EOM.linearize()
#print(EOM)
#EOM.(extraSubs=extraSubs)
#print(forcingToPy(EOM.q, EOM.F))
#print(EOM.toPython()) #extraSubs=extraSubs))
#model.linearize(noAcc=True, extraSubs=extraSubs)
#model.smallAngleSaveTex(folder='./_tex',prefix='test',variables=['MM','FF','M','C','K','B'])
#model.savePython(folder='./_py',prefix='test',variables=['MM','FF','M','C','K','B'], extraSubs=extraSubs)

In [ ]:
#model.exportPackage('_pkg')
x_B, z_B = symbols('x_B, z_B')
P_IMU = body.origin.locatenew('P_IMU', x_B * body.frame.x + z_B * body.frame.z) 
P_IMU = body.origin.locatenew('P_IMU', z_B * body.frame.z) 
#model.addPoint(P_IMU)
tilt=symbols('theta_t')

shaft_frame = ReferenceFrame('e_s')
shaft_frame.orient(body.frame, 'Body', (0,tilt,0), 'XYZ') 



P_IMU.pos_from(ref
               .origin)
P_IMU.vel(ref.frame)
P_IMU.acc(ref.frame)
P_IMU.acc(ref.frame).express(ref.frame)
P_IMU.acc(ref.frame).express(body.frame)
P_IMU.acc(ref.frame).express(shaft_frame)

In [ ]:
acc,accJac,Ma,Ca,Ka = model.computePointsMotion(noPosInJac=False)
Ma
Ca
Ka
accJac
#body.R_b2g
#body.omega_inertial
#model.computeBodiesMotion()

In [ ]:
from welib.yams.yams_sympy_tools import cleanPy, myjacobian

s0, params, inputs, sdofs = cleanPy(acc, dofs=model.q)
s0

In [ ]:
model.exportPackage('_pkg')

In [ ]:
P_0.acc(ref.frame)
P_O.acc(ref.frame)
P_0.vel(ref.frame)
P_O.vel(ref.frame)

P_0.pos_from(ref.origin)
P_0.pos_from(P_O)
P_0.pos_from(P_O).express(ref.frame)
P_O.pos_from(ref.origin)

In [ ]:
# --- Linearization of equation of motions
model.linearize(extraSubs=extraSubs, noVel=False)

model.M0
model.K0
model.C0
model.B0
model.var


In [ ]:
model.smallAngleApproxEOM([phi_x, phi_y, phi_z],order=1)
model.smallAngleLinearize(noVel=True)
model._sa_M
model._sa_K
model._sa_C
model._sa_B

In [ ]:
model.K0
#cleanPy(EOM.EOM, dofs=EOM.q, method='subs')
#cleanPy(EOM.EOM, dofs=EOM.q, method='string')
#EOM.EOM


In [ ]:
bd=model.body
ref=model.ref
#bd.masscenter_pos_global
#bd.masscenter_pos_global.express(ref.frame).subs(extraSubs)
#bd.R_b2g
bd.masscenter.vel(ref.frame)
#forcing
bd.frame.ang_vel_in(ref.frame)
#:bd.origin.vel(ref.frame)
#bd.frame.set_ang_vel(ref.frame, omega_x * e_T.x + omega_y * e_T.y  + omega_z * e_T.z)
#v=bd.masscenter.v2pt_theory(bd.origin, ref.frame, bd.frame) # GN & T are fixed in e_T
#v.simplify()
bd.inertia

In [ ]:
#from sympy import trigsimp
#mass_matrix = trigsimp(model.kane.mass_matrix)
#forcing = trigsimp(model.kane.forcing)

In [ ]:
# --- Getting EOM as function of coordinates and not omega
#model.kane.frstar
#model.kane.fr
#model.EOM
#model.body.inertia_matrix
#model.body.origin_inertia
vel=[diff(c,time) for c in model.coordinates]
acc=[diff(diff(c,time),time) for c in model.coordinates]
# --- Extract Mass Matrix and RHS from EOM
MM = - myjacobian(model.EOM(),acc)
MM
EOM =  model.EOM()# .doit().expand() # NOTE: doit is important
RHS = (MM * Matrix(acc) + model.EOM()).expand()
RHS = RHS.expand()
RHS

EOM = model.to_EOM()
print(EOM)
EOM.EOM

#MM_s = insertQQd(MM, coordinates)
#RHS_s = insertQQd(RHS, coordinates)
#
#CG = pend_COG.pos_from(Origin).to_matrix(inertial_frame)
#omega_loc = omega.to_matrix(pend_frame).simplify()

In [ ]:
model.kane.frstar
model.kane.mass_matrix_full
ff=model.kane.forcing_full
ff.simplify()
ff
#mass_matrix
#forcing

In [ ]:
# --- Full system mass matrix
#print(model.kane.mass_matrix.subs(extraSubs))

In [ ]:
# --- Small angle approximation mass matrix
#model._sa_mass_matrix

In [ ]:
#model.kane.forcing.subs(extraSubs)
#print(forcing)

In [ ]:
# --- Full system Forcing  RHS)
#model.kane.forcing.subs(extraSubs).subs(model.kdeqsSubs)

In [ ]:
# --- Small angle approximation forcing
model._sa_forcing

In [ ]:
# --- Equation of motions
#EOM=(model.fr+model.frstar).subs(model.kdeqsSubs)

In [ ]:
# --- Small angle Equation of motions
#model.smallAngleApproxEOM(model.smallAngles, extraSubs)
#model.EOM


In [ ]:
# --- display some info about bodies
model.body
#twr.bodyMassMatrix()